# 공공부문 AI 역량평가 문제 풀이 노트북

In [ ]:
# 공통 라이브러리 import
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, ConfusionMatrixDisplay

from sklearn.decomposition import PCA
from sklearn.cluster import KMeans

# 텍스트 분석용
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation

# 통계 분석용
from scipy.stats import pearsonr


## 3세트: 공공일자리 참여자 분석 (공공일자리_참여자_정보.csv)

In [ ]:
# 데이터 불러오기 (인코딩 및 오류 처리 추가)
import os

try:
    # 파일 경로 확인
    file_path = "공공일자리_참여자_정보.csv"
    if os.path.exists(file_path):
        df_job = pd.read_csv(file_path, encoding='utf-8')
    else:
        # 샘플 데이터 생성 (실제 파일이 없는 경우)
        print(f"파일을 찾을 수 없습니다: {file_path}")
        print("샘플 데이터를 생성합니다...")
        
        np.random.seed(42)
        sample_data = {
            '참여자_연령대': np.random.choice(['20대', '30대', '40대', '50대', '60대'], 1000),
            '참여자_성별': np.random.choice(['남성', '여성'], 1000),
            '교육_수준': np.random.choice(['고졸', '대졸', '대학원'], 1000),
            '참여_기간': np.random.randint(1, 12, 1000),
            '취업_연계_여부': np.random.choice([0, 1], 1000, p=[0.6, 0.4])
        }
        df_job = pd.DataFrame(sample_data)
    
    print(f"데이터 형태: {df_job.shape}")
    print("\n컬럼 정보:")
    print(df_job.info())
    print("\n데이터 미리보기:")
    print(df_job.head())
    
except Exception as e:
    print(f"데이터 로딩 중 오류 발생: {e}")
    df_job = None

In [ ]:
# 데이터 전처리 및 모델링 (오류 처리 추가)
if df_job is not None and not df_job.empty:
    try:
        # 결측값 확인 및 처리
        print("결측값 확인:")
        print(df_job.isnull().sum())
        
        # 결측값이 있다면 처리
        df_job = df_job.dropna()
        
        # 원-핫 인코딩 (숫자형 컬럼 제외)
        categorical_columns = df_job.select_dtypes(include=['object']).columns
        categorical_columns = [col for col in categorical_columns if col != '취업_연계_여부']
        
        if len(categorical_columns) > 0:
            df_encoded = pd.get_dummies(df_job, columns=categorical_columns, drop_first=True)
        else:
            df_encoded = df_job.copy()
        
        # 특성과 타겟 분리
        if '취업_연계_여부' in df_encoded.columns:
            X = df_encoded.drop('취업_연계_여부', axis=1)
            y = df_encoded['취업_연계_여부']
        else:
            print("타겟 변수 '취업_연계_여부'를 찾을 수 없습니다.")
            raise ValueError("타겟 변수가 없습니다.")
        
        print(f"특성 개수: {X.shape[1]}")
        print(f"샘플 개수: {X.shape[0]}")
        
        # train-test split
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
        
        # 로지스틱 회귀
        model = LogisticRegression(max_iter=1000, random_state=42)
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        
        # 성능 지표
        print("\\n=== 모델 성능 지표 ===")
        print(f"Accuracy: {accuracy_score(y_test, y_pred):.4f}")
        print(f"Precision: {precision_score(y_test, y_pred):.4f}")
        print(f"Recall: {recall_score(y_test, y_pred):.4f}")
        print(f"F1 Score: {f1_score(y_test, y_pred):.4f}")
        
        # 혼동 행렬 시각화
        plt.figure(figsize=(6, 4))
        cm = confusion_matrix(y_test, y_pred)
        disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=['미연계', '연계'])
        disp.plot(cmap='Blues')
        plt.title('취업 연계 예측 혼동 행렬')
        plt.tight_layout()
        plt.show()
        
        # 계수 해석 (상위 10개)
        print("\\n=== 특성 중요도 (상위 10개) ===")
        if len(X.columns) > 0:
            coeffs = pd.Series(model.coef_[0], index=X.columns)
            coeffs_sorted = coeffs.sort_values(ascending=False)
            print(coeffs_sorted.head(10))
            
            # 계수 시각화
            plt.figure(figsize=(10, 6))
            coeffs_sorted.head(10).plot(kind='barh')
            plt.title('특성별 중요도 (로지스틱 회귀 계수)')
            plt.xlabel('계수 값')
            plt.tight_layout()
            plt.show()
    
    except Exception as e:
        print(f"모델링 중 오류 발생: {e}")
        import traceback
        traceback.print_exc()
else:
    print("데이터가 로드되지 않았거나 비어있습니다.")

## 4세트: 정책토론 게시글 분석 (정책토론_게시글.csv)

In [ ]:
# 정책토론 게시글 데이터 로딩 (오류 처리 추가)
try:
    file_path = "정책토론_게시글.csv"
    if os.path.exists(file_path):
        df_forum = pd.read_csv(file_path, encoding='utf-8')
    else:
        print(f"파일을 찾을 수 없습니다: {file_path}")
        print("샘플 데이터를 생성합니다...")
        
        # 샘플 텍스트 데이터 생성
        sample_topics = [
            "교통 정책에 대한 의견입니다. 대중교통 확충이 필요합니다.",
            "교육 정책 개선 방안을 제안합니다. 학생 중심 교육이 중요합니다.",
            "환경 보호를 위한 정책이 시급합니다. 재생에너지 확대가 필요합니다.",
            "복지 정책 확대를 통해 사회적 약자를 보호해야 합니다.",
            "경제 활성화를 위한 중소기업 지원 정책이 필요합니다.",
            "의료 서비스 개선을 통해 국민 건강을 증진시켜야 합니다.",
            "주택 정책 개선으로 서민 주거 안정을 도모해야 합니다.",
            "문화 예술 지원 정책을 통해 문화 발전을 도모합시다.",
        ]
        
        np.random.seed(42)
        sample_data = {
            '제목': [f"정책 토론 {i+1}" for i in range(200)],
            '내용': np.random.choice(sample_topics, 200),
            '작성일': pd.date_range('2023-01-01', periods=200, freq='D')
        }
        df_forum = pd.DataFrame(sample_data)
    
    print(f"데이터 형태: {df_forum.shape}")
    print("\\n컬럼 정보:")
    print(df_forum.info())
    print("\\n데이터 미리보기:")
    print(df_forum.head())
    
except Exception as e:
    print(f"데이터 로딩 중 오류 발생: {e}")
    df_forum = None

In [ ]:
# 텍스트 분석 및 토픽 모델링 (오류 처리 추가)
if df_forum is not None and not df_forum.empty:
    try:
        # 텍스트 데이터 전처리
        if '내용' in df_forum.columns:
            # 결측값 제거
            df_forum_clean = df_forum.dropna(subset=['내용'])
            texts = df_forum_clean['내용'].astype(str)
            
            print(f"분석할 텍스트 개수: {len(texts)}")
            
            # 텍스트 벡터화
            vectorizer = CountVectorizer(
                max_df=0.95, 
                min_df=2, 
                stop_words=None,
                max_features=1000  # 최대 특성 수 제한
            )
            
            try:
                dtm = vectorizer.fit_transform(texts)
                print(f"벡터화 완료: {dtm.shape}")
                
                # LDA 토픽 모델링
                n_topics = min(5, dtm.shape[0] // 10)  # 데이터 크기에 따라 토픽 수 조정
                if n_topics < 2:
                    n_topics = 2
                    
                lda = LatentDirichletAllocation(
                    n_components=n_topics, 
                    random_state=42,
                    max_iter=10  # 반복 횟수 제한
                )
                lda.fit(dtm)
                
                # 토픽별 핵심 단어 추출
                n_top_words = 10
                feature_names = vectorizer.get_feature_names_out()
                
                print("\\n=== 토픽별 핵심 단어 ===")
                for topic_idx, topic in enumerate(lda.components_):
                    top_indices = topic.argsort()[:-n_top_words - 1:-1]
                    top_features = [feature_names[i] for i in top_indices]
                    print(f"토픽 {topic_idx+1}: {', '.join(top_features)}")
                
                # 토픽 분포 시각화
                plt.figure(figsize=(12, 6))
                
                # 문서별 토픽 분포
                doc_topic_probs = lda.transform(dtm)
                topic_avg = doc_topic_probs.mean(axis=0)
                
                plt.subplot(1, 2, 1)
                plt.bar(range(1, n_topics+1), topic_avg)
                plt.title('토픽별 평균 확률')
                plt.xlabel('토픽 번호')
                plt.ylabel('평균 확률')
                
                # 단어별 토픽 분포 (상위 토픽만)
                plt.subplot(1, 2, 2)
                top_topic_idx = np.argmax(topic_avg)
                top_topic = lda.components_[top_topic_idx]
                top_word_indices = top_topic.argsort()[-10:][::-1]
                top_words = [feature_names[i] for i in top_word_indices]
                top_probs = top_topic[top_word_indices]
                
                plt.barh(range(len(top_words)), top_probs)
                plt.yticks(range(len(top_words)), top_words)
                plt.title(f'토픽 {top_topic_idx+1}의 주요 단어')
                plt.xlabel('확률')
                
                plt.tight_layout()
                plt.show()
                
            except Exception as vec_error:
                print(f"벡터화 또는 토픽 모델링 오류: {vec_error}")
                print("단순 단어 빈도 분석을 수행합니다...")
                
                # 대체: 단순 단어 빈도 분석
                all_text = ' '.join(texts)
                words = all_text.split()
                word_freq = pd.Series(words).value_counts().head(20)
                
                plt.figure(figsize=(10, 6))
                word_freq.plot(kind='barh')
                plt.title('상위 20개 단어 빈도')
                plt.xlabel('빈도')
                plt.tight_layout()
                plt.show()
                
                print("\\n상위 20개 단어:")
                print(word_freq)
        else:
            print("'내용' 컬럼을 찾을 수 없습니다.")
            print(f"사용 가능한 컬럼: {list(df_forum.columns)}")
            
    except Exception as e:
        print(f"텍스트 분석 중 오류 발생: {e}")
        import traceback
        traceback.print_exc()
else:
    print("포럼 데이터가 로드되지 않았거나 비어있습니다.")

## 5세트: 농산물 가격 분석 (주요농산물_일별_가격동향.csv)

In [ ]:
# 농산물 가격 데이터 로딩 (오류 처리 추가)
try:
    file_path = "주요농산물_일별_가격동향.csv"
    if os.path.exists(file_path):
        df_crop = pd.read_csv(file_path, encoding='utf-8')
    else:
        print(f"파일을 찾을 수 없습니다: {file_path}")
        print("샘플 데이터를 생성합니다...")
        
        # 샘플 농산물 가격 데이터 생성
        np.random.seed(42)
        dates = pd.date_range('2023-01-01', periods=365, freq='D')
        
        sample_data = []
        for date in dates:
            # 배추 가격 (계절성 반영)
            base_cabbage = 2000 + 500 * np.sin(2 * np.pi * date.dayofyear / 365)
            cabbage_price = base_cabbage + np.random.normal(0, 200)
            
            # 사과 가격 (다른 패턴)
            base_apple = 3000 + 300 * np.cos(2 * np.pi * date.dayofyear / 365)
            apple_price = base_apple + np.random.normal(0, 150)
            
            sample_data.append({
                '날짜': date.strftime('%Y-%m-%d'),
                '품목': '배추',
                '평균가격': max(cabbage_price, 500)  # 최소 가격 보장
            })
            sample_data.append({
                '날짜': date.strftime('%Y-%m-%d'),
                '품목': '사과',
                '평균가격': max(apple_price, 1000)  # 최소 가격 보장
            })
        
        df_crop = pd.DataFrame(sample_data)
    
    print(f"데이터 형태: {df_crop.shape}")
    print("\\n컬럼 정보:")
    print(df_crop.info())
    print("\\n데이터 미리보기:")
    print(df_crop.head())
    print("\\n품목별 데이터 개수:")
    if '품목' in df_crop.columns:
        print(df_crop['품목'].value_counts())
    
except Exception as e:
    print(f"데이터 로딩 중 오류 발생: {e}")
    df_crop = None

In [ ]:
# 농산물 가격 상관관계 분석 (오류 처리 추가)
if df_crop is not None and not df_crop.empty:
    try:
        # 필요한 품목이 있는지 확인
        available_items = df_crop['품목'].unique() if '품목' in df_crop.columns else []
        target_items = ['배추', '사과']
        
        print(f"사용 가능한 품목: {available_items}")
        print(f"분석 대상 품목: {target_items}")
        
        # 대상 품목이 있는지 확인
        existing_items = [item for item in target_items if item in available_items]
        
        if len(existing_items) >= 2:
            # 배추, 사과 데이터 필터링 및 피벗
            df_filtered = df_crop[df_crop['품목'].isin(existing_items)].copy()
            
            # 날짜 컬럼 처리
            if '날짜' in df_filtered.columns:
                df_filtered['날짜'] = pd.to_datetime(df_filtered['날짜'])
                df_filtered = df_filtered.sort_values('날짜')
            
            # 피벗 테이블 생성
            pivot = df_filtered.pivot(index='날짜', columns='품목', values='평균가격')
            print(f"\\n피벗 테이블 형태: {pivot.shape}")
            print(pivot.head())
            
            # 결측값 확인 및 처리
            print("\\n결측값 개수:")
            print(pivot.isnull().sum())
            
            if len(existing_items) == 2:
                item1, item2 = existing_items
                
                # 공통 날짜의 데이터만 사용
                common_data = pivot.dropna()
                print(f"\\n공통 데이터 개수: {len(common_data)}")
                
                if len(common_data) > 2:
                    # 상관관계 계산
                    corr, pval = pearsonr(common_data[item1], common_data[item2])
                    print(f"\\n=== {item1} vs {item2} 상관관계 ===")
                    print(f"Correlation: {corr:.4f}")
                    print(f"p-value: {pval:.4f}")
                    print(f"유의성: {'유의함' if pval < 0.05 else '유의하지 않음'} (α=0.05)")
                    
                    # 시각화
                    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 10))
                    
                    # 시계열 그래프
                    ax1_twin = ax1.twinx()
                    
                    line1 = ax1.plot(pivot.index, pivot[item1], label=f'{item1}', color='green', linewidth=2)
                    line2 = ax1_twin.plot(pivot.index, pivot[item2], label=f'{item2}', color='red', linewidth=2)
                    
                    ax1.set_ylabel(f'{item1} 가격 (원)', color='green')
                    ax1_twin.set_ylabel(f'{item2} 가격 (원)', color='red')
                    ax1.set_title(f'{item1} vs {item2} 가격 추이 (상관계수: {corr:.3f})')
                    ax1.grid(True, alpha=0.3)
                    
                    # 범례 설정
                    lines = line1 + line2
                    labels = [l.get_label() for l in lines]
                    ax1.legend(lines, labels, loc='upper left')
                    
                    # 산점도
                    ax2.scatter(common_data[item1], common_data[item2], alpha=0.6, s=30)
                    ax2.set_xlabel(f'{item1} 가격 (원)')
                    ax2.set_ylabel(f'{item2} 가격 (원)')
                    ax2.set_title(f'{item1} vs {item2} 산점도')
                    ax2.grid(True, alpha=0.3)
                    
                    # 추세선 추가
                    z = np.polyfit(common_data[item1], common_data[item2], 1)
                    p = np.poly1d(z)
                    ax2.plot(common_data[item1], p(common_data[item1]), 
                            "r--", alpha=0.8, label=f'추세선 (기울기: {z[0]:.3f})')
                    ax2.legend()
                    
                    plt.tight_layout()
                    plt.show()
                    
                    # 추가 통계 정보
                    print(f"\\n=== 추가 통계 정보 ===")
                    print(f"{item1} - 평균: {common_data[item1].mean():.2f}, 표준편차: {common_data[item1].std():.2f}")
                    print(f"{item2} - 평균: {common_data[item2].mean():.2f}, 표준편차: {common_data[item2].std():.2f}")
                    
                else:
                    print("상관관계 분석을 위한 데이터가 충분하지 않습니다.")
            else:
                print("분석을 위해 2개의 품목이 필요합니다.")
                
        else:
            print(f"분석 대상 품목({target_items})을 찾을 수 없습니다.")
            print(f"대신 사용 가능한 품목들의 가격 추이를 보여드립니다.")
            
            # 사용 가능한 품목들의 가격 추이
            if len(available_items) > 0:
                plt.figure(figsize=(12, 6))
                for item in available_items[:5]:  # 최대 5개 품목만
                    item_data = df_crop[df_crop['품목'] == item]
                    if '날짜' in item_data.columns:
                        item_data['날짜'] = pd.to_datetime(item_data['날짜'])
                        item_data = item_data.sort_values('날짜')
                    plt.plot(item_data['날짜'], item_data['평균가격'], label=item, linewidth=2)
                
                plt.title('농산물 가격 추이')
                plt.xlabel('날짜')
                plt.ylabel('평균가격 (원)')
                plt.legend()
                plt.grid(True, alpha=0.3)
                plt.xticks(rotation=45)
                plt.tight_layout()
                plt.show()
    
    except Exception as e:
        print(f"농산물 가격 분석 중 오류 발생: {e}")
        import traceback
        traceback.print_exc()
else:
    print("농산물 가격 데이터가 로드되지 않았거나 비어있습니다.")

## 6세트: 감염병 신고 현황 분석 (감염병_신고현황.csv)

In [ ]:
# 감염병 신고 데이터 로딩 (오류 처리 추가)
try:
    file_path = "감염병_신고현황.csv"
    if os.path.exists(file_path):
        df_disease = pd.read_csv(file_path, encoding='utf-8')
    else:
        print(f"파일을 찾을 수 없습니다: {file_path}")
        print("샘플 데이터를 생성합니다...")
        
        # 샘플 감염병 데이터 생성
        np.random.seed(42)
        
        regions = ['서울특별시', '부산광역시', '대구광역시', '인천광역시', '광주광역시', 
                  '대전광역시', '울산광역시', '세종특별자치시', '경기도', '강원도',
                  '충청북도', '충청남도', '전라북도', '전라남도', '경상북도', '경상남도', '제주특별자치도']
        
        diseases = ['인플루엔자', '코로나19', '결핵', '수두', '유행성이하선염', 
                   '백일해', '홍역', 'A형간염', '장티푸스', '세균성이질']
        
        age_groups = ['0-9세', '10-19세', '20-29세', '30-39세', '40-49세', 
                     '50-59세', '60-69세', '70-79세', '80세이상']
        
        sample_data = []
        for _ in range(1000):
            sample_data.append({
                '지역': np.random.choice(regions),
                '감염병명': np.random.choice(diseases),
                '확진자_연령': np.random.choice(age_groups),
                '확진자_수': np.random.randint(1, 100),
                '신고일': pd.to_datetime('2023-01-01') + pd.Timedelta(days=np.random.randint(0, 365))
            })
        
        df_disease = pd.DataFrame(sample_data)
    
    print(f"데이터 형태: {df_disease.shape}")
    print("\\n컬럼 정보:")
    print(df_disease.info())
    print("\\n데이터 미리보기:")
    print(df_disease.head())
    
    if '지역' in df_disease.columns:
        print("\\n지역별 데이터 개수:")
        print(df_disease['지역'].value_counts().head(10))
    
    if '감염병명' in df_disease.columns:
        print("\\n감염병별 데이터 개수:")
        print(df_disease['감염병명'].value_counts())
    
except Exception as e:
    print(f"데이터 로딩 중 오류 발생: {e}")
    df_disease = None

In [ ]:
# 감염병 데이터 군집 분석 (오류 처리 추가)
if df_disease is not None and not df_disease.empty:
    try:
        # 데이터 전처리
        df_analysis = df_disease.copy()
        
        # 지역 단위 변환 (시/군 → 시/도)
        if '지역' in df_analysis.columns:
            # 광역시/도 단위로 변환
            df_analysis['시도'] = df_analysis['지역'].str.extract(r'^([가-힣]+[시도]|[가-힣]+특별[시자치시]|[가-힣]+광역시)')
            df_analysis['시도'] = df_analysis['시도'].fillna(df_analysis['지역'].str[:2])
        else:
            print("'지역' 컬럼을 찾을 수 없습니다.")
            df_analysis['시도'] = '기타'
        
        print("시도별 데이터 분포:")
        print(df_analysis['시도'].value_counts())
        
        # 피처 선택 및 전처리
        features_to_encode = []
        if '시도' in df_analysis.columns:
            features_to_encode.append('시도')
        if '감염병명' in df_analysis.columns:
            features_to_encode.append('감염병명')
        if '확진자_연령' in df_analysis.columns:
            features_to_encode.append('확진자_연령')
        
        if len(features_to_encode) == 0:
            print("인코딩할 피처가 없습니다.")
            raise ValueError("분석할 수 있는 범주형 변수가 없습니다.")
        
        print(f"\\n인코딩할 피처: {features_to_encode}")
        
        # 원-핫 인코딩
        X = pd.get_dummies(df_analysis[features_to_encode], drop_first=True)
        print(f"인코딩 후 피처 수: {X.shape[1]}")
        print(f"샘플 수: {X.shape[0]}")
        
        if X.shape[1] == 0 or X.shape[0] < 4:
            print("군집 분석을 위한 데이터가 충분하지 않습니다.")
            raise ValueError("데이터가 부족합니다.")
        
        # 최적 군집 수 결정 (엘보우 방법)
        max_clusters = min(10, X.shape[0] // 2)
        inertias = []
        cluster_range = range(2, max_clusters + 1)
        
        print(f"\\n최적 군집 수 탐색 (2~{max_clusters}개)")
        for k in cluster_range:
            kmeans_temp = KMeans(n_clusters=k, random_state=42, n_init=10)
            kmeans_temp.fit(X)
            inertias.append(kmeans_temp.inertia_)
        
        # 엘보우 방법 시각화
        plt.figure(figsize=(15, 5))
        
        plt.subplot(1, 3, 1)
        plt.plot(cluster_range, inertias, 'bo-')
        plt.xlabel('군집 수')
        plt.ylabel('Inertia')
        plt.title('엘보우 방법을 이용한 최적 군집 수')
        plt.grid(True, alpha=0.3)
        
        # 최적 군집 수 선택 (엘보우 포인트 추정)
        if len(inertias) >= 3:
            # 2차 차분을 이용한 엘보우 포인트 찾기
            diffs = np.diff(inertias)
            second_diffs = np.diff(diffs)
            if len(second_diffs) > 0:
                optimal_k = np.argmax(second_diffs) + 3  # +3은 인덱스 보정
                optimal_k = min(optimal_k, max_clusters)
            else:
                optimal_k = 4
        else:
            optimal_k = 4
        
        print(f"선택된 군집 수: {optimal_k}")
        
        # K-means 군집화
        kmeans = KMeans(n_clusters=optimal_k, random_state=42, n_init=10)
        clusters = kmeans.fit_predict(X)
        df_analysis['cluster'] = clusters
        
        print(f"\\n군집별 데이터 분포:")
        print(pd.Series(clusters).value_counts().sort_index())
        
        # PCA 시각화 (2D)
        if X.shape[1] >= 2:
            pca = PCA(n_components=2)
            X_pca = pca.fit_transform(X)
            
            explained_variance = pca.explained_variance_ratio_
            print(f"\\nPCA 설명 분산 비율: {explained_variance}")
            print(f"총 설명 분산: {explained_variance.sum():.3f}")
            
            plt.subplot(1, 3, 2)
            scatter = plt.scatter(X_pca[:, 0], X_pca[:, 1], c=clusters, cmap='rainbow', alpha=0.7)
            plt.colorbar(scatter)
            plt.xlabel(f'PC1 ({explained_variance[0]:.2%})')
            plt.ylabel(f'PC2 ({explained_variance[1]:.2%})')
            plt.title('감염병 패턴 군집화 (PCA 2D)')
            plt.grid(True, alpha=0.3)
            
            # 군집 중심점 표시
            centers_pca = pca.transform(kmeans.cluster_centers_)
            plt.scatter(centers_pca[:, 0], centers_pca[:, 1], 
                       c='black', marker='x', s=200, linewidths=3, label='중심점')
            plt.legend()
        
        # 군집별 특성 분석
        plt.subplot(1, 3, 3)
        cluster_counts = pd.Series(clusters).value_counts().sort_index()
        plt.bar(cluster_counts.index, cluster_counts.values)
        plt.xlabel('군집 번호')
        plt.ylabel('데이터 개수')
        plt.title('군집별 데이터 분포')
        plt.grid(True, alpha=0.3)
        
        plt.tight_layout()
        plt.show()
        
        # 군집별 상세 분석
        print("\\n=== 군집별 특성 분석 ===")
        for cluster_id in sorted(df_analysis['cluster'].unique()):
            cluster_data = df_analysis[df_analysis['cluster'] == cluster_id]
            print(f"\\n군집 {cluster_id} (총 {len(cluster_data)}개)")
            
            # 주요 특성별 분포
            for feature in features_to_encode:
                if feature in cluster_data.columns:
                    top_values = cluster_data[feature].value_counts().head(3)
                    print(f"  {feature}: {dict(top_values)}")
        
        # 실루엣 점수 계산
        from sklearn.metrics import silhouette_score
        if len(set(clusters)) > 1 and len(set(clusters)) < len(clusters):
            sil_score = silhouette_score(X, clusters)
            print(f"\\n실루엣 점수: {sil_score:.3f}")
            print("실루엣 점수 해석:")
            if sil_score > 0.7:
                print("  - 매우 좋은 군집화")
            elif sil_score > 0.5:
                print("  - 좋은 군집화")
            elif sil_score > 0.25:
                print("  - 보통 군집화")
            else:
                print("  - 개선이 필요한 군집화")
        
    except Exception as e:
        print(f"군집 분석 중 오류 발생: {e}")
        import traceback
        traceback.print_exc()
else:
    print("감염병 데이터가 로드되지 않았거나 비어있습니다.")